# Evaluate Neural Network Robustness against Adversarial Attacks

**18-819 Project, Fall 2022**

Team Members:

- Claudio Gomes (cpratago@andrew.cmu.edu)
- Ke Xu (kx1@andrew.cmu.edu)
- Yuxuan Zheng (yzheng5@andrew.cmu.edu)

## Introduction

### Background

Neural Networks are already widely used in many areas, such as image classification, language understanding, game playing, and self-driving cars.
Although neural networks help to solve many traditionally hard problems as a universal function approximator,
they are unfortunately vulnerable to the adversarial attacks of its inputs.

Adversarial Attacks use slightly perturbed inputs from the dataset and attempt to fool the pre-trained neural network to make a wrong prediction.

For example, as shown in Figure 1, after adding a slight noise to the original image, the network alters its prediction from the correct "panda" to the incorrect "gibbon".
Also note that, the human eyes cannot distinguish the original and perturbed images because they both look like a panda!

<p align="center">
  <img src="pics/adver_example_1.jpg">
</p>
<center>Figure 1. An Example of Adversarial Attack</center>

Such attacks expose great risk to leverage neural networks in our daily life.
Imagine that if a slightly perturbed "stop sign" image may be incorrectly interpreted by the self-driving cars,
the use of such technology becomes risky and controversial.

For a pre-trained neural network model, if it is possible to make such a wrong prediction with a slightly altered input,
we say this model has an **adversarial example**; or, there exists a possible **adversarial attack**.

Therefore, in our project, we want to evaluate the given pre-trained neural network to verify that it is robust enough under such perturbed images.
In other words, we want to check if our model has an adversarial example.

### Specific Network in Our Project

In our project, we focus on a typical 3-layer fully-connected deep neural network with `ReLU` activation layers.
The model structure is shown in Figure 2.

<p align="center">
  <img src="pics/nn_structure.jpg">
</p>
<center>Figure 2. Neural Network to Evaluate</center>

The model was pre-trained with the mnist dataset and the trained parameters are saved in the `xu_net_for_MNIST.pth` file that will be loaded upon predicting the label of a given image.

## Problem Math Formulation

In this section, we describe how to convert the goal of evaluating the pre-trained neural network model into a math problem,
specifically, a Mixed-Integer Linear Programming (MILP) optimization problem.

### Overview

Given a pre-defined perturbation range of the input image, we want to know the **worst prediction** our model may make.
In other words, we want to **maximize the possibility to make a wrong prediction with the perturbed image**.

Fundamentally, the neural network will make the prediction based on the last layer's outputs,
because the last layer will output a probability distribution over all possible image labels.
In our network, the model will pick the label that has the largest value among its last layer's outputs.
Specifically, since we are using the mnist dataset with 10 classes, the last layer will have 10 neurons and the prediction
is the label that has highest value among those 10 neurons.

Therefore, if the model's last layer still has the same largest output given the faked image as the true image,
we know that the model will not make a wrong prediction under such altered images (within the pre-defined perturbation range).

With that, our optimization problem is to maximize the possibility to make a wrong prediction, given the possible neuron activations range as the constraints.
And after that, we can check if the model can still output the correct prediction under such a terrible attack.

### Steps

There are two phases of constructing such a optimization problem:

1. Go through each layer of the neural network to add constraints to the final optimization problem.
   1. When we see a `Linear` layer, we calculate the lower and upper bound of the activations of this `Linear` layer. This is done by solve two smaller LP problems:
      1. To get the lower bound of activations, we want to **minimize** the activation values given the previous layer activations (variables within ranges) and pre-loaded model weights and biases.
      2. To get the upper bound of activations, we simply **maximize** the activation values given the same constraints.
   2. When we see a `ReLU` layer, we append the constraints introduced by this `ReLU` layer to the constraint set of the final optimization problem.
      1. The details of the appended constraints are explained in the section below.
2. Solve the final optimization problem after we went through the model layer-by-layer.
   1. The final optimization has an objective to **minimize the difference of the output activation between the correct class and a wrong class**.
   2. Since the model makes the prediction based on the highest output activations (for example, if class 2 has the highest output activation, we predict the image as class 2), if we minimize the difference of the output activations between the correct and wrong label, we are trying to force the model to make a *very wrong* prediction under the perturbed input.
   3. Then, the optimization problem aligns with our goal to maximize the possibility to make a wrong prediction.

After solving the final optimization problem, we know the perturbed input image that makes the model to make the worst prediction.
If the model can still make the correct prediction given such perturbed input image, we know the model has no adversarial example.

For example, we can see the output of our classical solver:

```
2022-11-30 10:24:39 | classical | INFO: Prediction given the original image: class 7
2022-11-30 10:24:39 | classical | INFO: Prediction given the worst perturbed image: class 7
2022-11-30 10:24:39 | classical | INFO: No adversarial input exists for this model.
```

It means that the model will predict a true image as class 7, and it will predict the worst-perturbed image as class 7 as well.
So, it doesn't have an adversarial example.
And we can claim that we evaluated the model and it is robust under this specific attack.

### Model `ReLU` Layer as Constraints

TODO: Ke

## Environment Setup

Before advancing to the next steps, we need to verify if the environment is correctly set up. There are 5 requirements:

1. [CVXPY](https://www.cvxpy.org/) -> Open-source Python-embedded modeling language for convex optimization problems.
2. [CVXOPT](https://cvxopt.org/) -> Open-source software package for convex optimization.
3. [IBM ILOG CPLEX Optimization Studio](https://www.ibm.com/products/ilog-cplex-optimization-studio) -> Software package that contains the CPLEX optimizer.
4. [D-Wave System](https://docs.ocean.dwavesys.com/projects/system/en/latest/) -> API that enables us to use the D-Wave systems to solve problems.
5. [D-Wave Hybrid](https://docs.ocean.dwavesys.com/projects/hybrid/en/latest/) -> API that enables us to build hybrid asynchronous decomposition samplers to solve large problems.

### CVXPY and CVXOPT

Installing `CVXPY` and `CVXOPT` is straightforward.
We use `CVXPY` to model the different LPs in our problem.
`CVXOPT` is used to solve these LPs.

In [ ]:
# Install CVXPY and CVXOPT
%pip install cvxpy
%pip install cvxopt

### IBM ILOG CPLEX Optimization Studio

This software is not quite straightforward to install.
Fortunately, the software is provided at no cost for academic users.

Firstly, we go to the [main page](https://www.ibm.com/products/ilog-cplex-optimization-studio) and click on the button ***Get the no-cost academic edition***.
This button will redirect you to a [Data Science page](https://www.ibm.com/academic/topic/data-science) where you have to register using our academic email (for example, `{cmu_id}@andrew.cmu.edu`).

Next, after registering, we return to the [Data Science page](https://www.ibm.com/academic/topic/data-science) and scroll down to a list that shows *courseware*.
We click the button that changes the list to show *software*.
On the software list, IBM ILOG CPLEX Optimization Studio is shown.
We click on it and follow the download instructions (***note: we suggest installing it using the HTTP donwload option to avoid installing their middleware donwloader***).

During the installation, we have to make sure to know in which path will the software be installed,
as it is important for setting the Python API.
Although not strictly required, we make sure to let the installer add the program to the `PATH`.

Once the program is installed, we follow the steps shown in this [page](https://www.ibm.com/docs/en/icos/12.8.0.0?topic=cplex-setting-up-python-api).
Most of the times, it is sufficient to execute the following command:

```
python setup.py install
```

Finally, we verify that `CPLEX` is working as intended by running the following command:

In [4]:
import cplex

If the command worked without errors, `CVXPY` will be able to use the `CPLEX` solver to solve problems.

### D-Wave System

To install D-Wave's Python API to access their quantum systems, we execute the following command:

In [ ]:
# Install D-Wave's Python API to access cloud services
%pip install dwave-system

After installing, we need to setup the API with our API token.
First, to retrieve the API token, we access our D-Wave Leap account dashboard.
There, a button to copy our API token is visible.

Once we have our API token copied, we run the following command.
We can just press enter for all the prompts, minus the prompt for the API token.
We have to make sure that we give the API token at this point.

```
dwave setup
```

To confirm that this API has been correctly installed and setup, we run the following command:

In [13]:
! dwave ping -s Advantage_system4.1

Using endpoint: https://na-west-1.cloud.dwavesys.com/sapi/v2/
Using region: na-west-1
Using solver: Advantage_system4.1
Submitted problem ID: f10d8578-b112-47b0-9f52-f99e1ddafcf0

Wall clock time:
 * Solver definition fetch: 1451.468 ms
 * Problem submit and results fetch: 1383.979 ms
 * Total: 2835.447 ms

QPU timing:
 * post_processing_overhead_time = 658.0 us
 * qpu_access_overhead_time = 1842.17 us
 * qpu_access_time = 15854.83 us
 * qpu_anneal_time_per_sample = 20.0 us
 * qpu_delay_time_per_sample = 20.54 us
 * qpu_programming_time = 15777.97 us
 * qpu_readout_time_per_sample = 36.32 us
 * qpu_sampling_time = 76.86 us
 * total_post_processing_time = 658.0 us


The ping command above should be successful, assuming D-Wave's solvers are not down.
It should display the QPU timings such as the readout time per sample, in microseconds.

### D-Wave Hybrid

Finally, we have to install dwave-hybrid:

In [14]:
# Install D-Wave Hybrid.
%pip install dwave-hybrid


     ---------------------------------------- 74.7/74.7 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 16.2 MB/s eta 0:00:00


## Classical Approach

Now, you are able to run the classical solver on CPU with the following command.
The code `classical/solver_mip.py` constructs and solves the MILP problem explained in the math formulation section above.

In [2]:
## Uncomment if you want to use CPLEX and generate .lp file of the model.
## By default, it uses CVXOPT.
## Args '-C' or '--CPLEX' make the function use CPLEX and generate .lp file.
!python classical/solver_mip.py -C

2022-12-05 11:03:52 | classical | INFO: Loading the pre-trained neural network model...
2022-12-05 11:03:52 | classical | INFO: Loading the model parameters...
2022-12-05 11:03:52 | classical | INFO: Constructing the math program to solve...
2022-12-05 11:03:52 | classical | INFO: 
2022-12-05 11:03:52 | classical | INFO: Stage 1: solve lower/upper bounds of Linear layers and append constraints for ReLU layers...
2022-12-05 11:03:52 | classical | INFO: Considering: linear_relu_stack.0, layer: Linear(in_features=784, out_features=20, bias=True)
2022-12-05 11:03:52 | classical | INFO:   lower bound: 
[[-0.4234205800365508 -1.8351959504381299 0.8804067259743539
  -0.8196989746911543 -3.793728826809773 -1.8864171617425003
  -0.5739344345791544 0.8865537540367369 -1.6154333620130206
  -0.08706178086480955 -2.3534664022913234 -2.0912231290293244
  -1.7001457389483525 -2.3174443445543202 -0.24076084230795802
  0.11701600115205879 -2.1210138638766782 -1.9108620069399005
  -3.217863043469488 10.

## Quantum Approach

Here, we solve the same MILP problem with the hybrid solver provided by D-Wave.

### First issue to take into consideration: Continuous Variables

The problem contains continuous variables, which require encoding techniques so that quantum machines can represent them with a given precision level.

<p align="center">
  <img src="pics/encoding.png">
</p>
<center>Figure Ref.: [^2], Zhang, R., Lu, D. & Yin, H. A Generalized Floating-Point Representation and Manipulation of Quantum Signals Based on IEEE-754. Int J Theor Phys 59, 936–952 (2020). [https://doi.org/10.1007/s10773-019-04379-y](https://doi.org/10.1007/s10773-019-04379-y)</center>

<!-- ![Encoding Continuous Numbers](pics/encoding.png) -->


The encoding requires lots of qubits. For instance, if we want to use a precision of 32 bits for this encoding system to represent the 824 continuous variables of our LP, then ***at least 26,368 qubits are needed!*** In other words, our toy model is already too complex for today's quantum machines.

Moreover, when performing operations on these numbers, we also face an increased number of gates that exceeds the capabilities of today's quantum machines. The following figure is an example of a circuit that multiplies a continuous number by 2.

<p align="center">
  <img width=50% src="pics/encodingmult.png">
</p>
<center>Figure Ref.: [^2], Zhang, R., Lu, D. & Yin, H. A Generalized Floating-Point Representation and Manipulation of Quantum Signals Based on IEEE-754. Int J Theor Phys 59, 936–952 (2020). [https://doi.org/10.1007/s10773-019-04379-y](https://doi.org/10.1007/s10773-019-04379-y)</center>

<!-- ![Multiply by 2 of continuous numbers](pics/encodingmult.png) -->

### Second issue to take into consideration: Numerous Constraints

Our LP has two constraints for each pixel of the perturbation, since each pixel is constrained to be between -0.1 and 0.1. Moreover, each `ReLU` adds 4 constraints per neuron to the LP. This means that our toy LP model has $784⨉2 + 4⨉20⨉2 = 1728$ constraints!

Mapping these constraints into the quantum machine is a difficult tasks, requiring many qubits and operations. Unfortunately, this is another issue that overcomes the capabilities of today's quantum machines.

### How to tackle those issues?

We have checked the quantum machines that are provided by many providers, including IBM, Rigetti and D-Wave. Unfortunately, as expected, none of these quantum machines are sufficiently big to handle this LP problem, much less robust.

Fortunately, D-Wave launched in late 2021 a powerful hybrid solver that decomposes and solves large-scale problems. The solver was later updated in May 2022 to support real numbers and recently in November 2022 to support much more complex problems. This solver uses very advanced decomposition techniques and is called via D-Wave’s Python API with the `LeapHybridCQMSampler` class. ***It supports 1,000,000 variables (including continuous) and 100,000 constraints, enabling it to solve real industry-sized problems.***

### Constrained Quadratic Model

D-Wave's Hybrid CQM Solver solves problems formulated as Constrained Quadratic Models (CQMs). This formulation is represented as follows:

<p align="center">
  <img width=50% src="pics/cqmformulation.png">
</p>

<!-- ![Constrained Quadratic Model](pics/cqmformulation.png) -->

This enables us to directly add the constraints instead of having to converting them into the objective function.

### `dimod` library

To model a problem as a CQM, we use D-Wave's `dimod` library, which provides functions to create CQMs. One of these functions is `cqm = dimod.lp.load('example.lp')`, which constructs a CQM from a LP file.

An LP file is a file that stores a linear problem model. This type of file can be generated by the `CPLEX` optimizer.
We used the `CPLEX` optimizer to solve the `CVXPY` model and output it as a LP file. This LP file is then read by `cqm = dimod.lp.load('example.lp')`, enabling us to solve the same problem using D-Wave's hybrid solver.

### D-Wave System library

Now that we have the CQM, we can solve it using D-Wave's hybrid solver:

```
cqm = dimod.lp.load(f)
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm, time_limit=5)
```

D-Wave's hybrid solver is called using the `LeapHybridCQMSampler` class and its function `sample_cqm()`.

### How it works

Unfortunately, as mentioned by D-Wave in a recent reply ([link](https://support.dwavesys.com/hc/en-us/community/posts/9368364472855-Doubts-on-the-Leap-hybrid-CQM-solver)),
the internal functioning of the `LeapHybridCQMSampler` class and its function `sample_cqm()` is proprietary and not publicly available yet.

Nonetheless, D-Wave published some white papers and technical reports that give some insight and hints on the internal functioning [[White Paper Link](https://www.dwavesys.com/media/rldh2ghw/14-1055a-a_hybrid_solver_for_constrained_quadratic_models.pdf), [Technical Report Link](https://www.dwavesys.com/media/0ftjce4l/measuring-performance-of-the-leap-constrained-quadratic-model-solver.pdf)].

In these papers, they present a good diagram of the process behind the hybrid solver:

<p align="center">
  <img width=50% src="pics/diagram.png">
</p>

<!-- ![Diagram](pics/diagram.png) -->

In short, the hybrid solver runs several heuristics in parallel. Some of them are classical, and some others are quantum. It then keeps iterating on each heuristic until the time limit T is reached.

### D-Wave Hybrid Library

In these papers, D-Wave also hints at their `hybrid` library, which appears to be the "skeleton" of the hybrid solver.
This library enables the creation of hybrid asynchronous decomposition samplers.
That is, samplers that decompose a given problem in many smaller subproblems, and that use classical and quantum heuristics to solve it.

When looking at the documentation ([link](https://docs.ocean.dwavesys.com/projects/hybrid/en/latest/intro/overview.html)), we can observe a diagram that is a good illustration of how the library works:

<p align="center">
  <img width=50% src="pics/HybridBlockDiagram.png">
</p>

<!-- ![HybridBlockDiagram](pics/HybridBlockDiagram.png) -->

Alternatively, in terms of code, we can quickly implement these workflows as follows:

```
import dimod
import hybrid

# Construct a problem
bqm = dimod.BinaryQuadraticModel({}, {'ab': 1, 'bc': -1, 'ca': 1}, 0, dimod.SPIN)

# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

# Solve the problem
init_state = hybrid.State.from_problem(bqm)
final_state = workflow.run(init_state).result()

# Print results
print("Solution: sample={.samples.first}".format(final_state))
```

We can choose which heuristics to use, as well as which decomposition techniques to use.
Moreover, custom blocks can be created and incorporated in these workflows.
The library enables us to easily incorporate our own classical heuristic ou decomposer in these workflows.

However, as the reader may have already noticed, the library contains no reference to CQMs.
Unfortunately, it seems that D-Wave has not updated this library with code to solve CQMs yet:

<p align="center">
  <img width=50% src="pics/codeproprietary.png">
</p>

<!-- ![codeproprietary](pics/codeproprietary.png) -->

### Script to solve the problem using D-Wave's Hybrid Solver.

The code `quantum/solver_dwave.py` reads the LP file, generates a CQM, and solves it using D-Wave's hybrid solver.

In [ ]:
## Uncomment if you want to calculate a new sampleset.
## By default, it uses last obtained sampleset.
## Args '-s' or '--sampleset' make the function calculate new sampleset.
# !python quantum/solver_dwave.py -s
!python quantum/solver_dwave.py

2022-11-25 20:11:46 | main | DEBUG: Received input argument: Namespace(verbose=True, sampleset=False)
2022-11-25 20:11:46 | main | DEBUG: Project path: C:\Users\Work\Documents\GitHub\18819-project
2022-11-25 20:11:46 | main | INFO: 43 feasible solutions of 62.
2022-11-25 20:11:46 | main | INFO: Best sample: Sample(sample={'x1': -0.0, 'x10': 1.2686546325486001, 'x100': 0.1, 'x101': 0.0, 'x102': 0.0, 'x103': 0.0, 'x104': 0.1, 'x105': 0.0, 'x106': 0.0, 'x107': 0.09999999999999999, 'x108': 0.0, 'x109': 0.0, 'x11': 1.8706970922190018, 'x110': 0.0, 'x111': 0.1, 'x112': 0.0, 'x113': 0.1, 'x114': 0.1, 'x115': 0.1, 'x116': 0.1, 'x117': 0.1, 'x118': 0.09999999999999999, 'x119': 0.09999999999999999, 'x12': 2.7112278645721823, 'x120': 0.1, 'x121': 0.1, 'x122': 0.1, 'x123': 0.1, 'x124': 0.1, 'x125': 0.0, 'x126': 0.0, 'x127': 0.0, 'x128': 0.0, 'x129': 0.0, 'x13': -0.0, 'x130': 0.0, 'x131': 0.0, 'x132': 0.0, 'x133': 0.0, 'x134': 0.0, 'x135': 0.0, 'x136': 0.1, 'x137': 0.0, 'x138': 0.0, 'x139': 0.1, 'x

### New Run of Quantum Approach to Avoid Erasing the Output [To be deleted]

In [ ]:
# !python quantum/solver_dwave.py -s
!python quantum/solver_dwave.py

2022-11-30 10:39:36 | quantum | INFO: Loading the pre-trained neural network model...
/home/eric/Apps/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
2022-11-30 10:39:36 | quantum | INFO: 
2022-11-30 10:39:36 | quantum | INFO: Solving the final LP with D-Wave...
2022-11-30 10:39:36 | quantum | INFO: ...done
2022-11-30 10:39:36 | quantum | INFO: 43 feasible solutions of 62.
2022-11-30 10:39:36 | quantum | INFO

## Comparison

In this section, we compare the result correctness and execution performance of the classical and quantum solver.
We used the `CPLEX` optimizer to find 50 images with known adversarial examples.
We passed these 50 images to the D-Wave Hybrid Solver to find out if it also finds adversarial examples for all of these images, using a time limit of 5 seconds:

In [6]:
!python quantum/solver_dwave_images_found.py

2022-12-05 11:23:12 | classical | DEBUG: Received input argument: Namespace(verbose=True)
2022-12-05 11:23:12 | classical | INFO: No adversarial input was found for this model for image4.
2022-12-05 11:23:12 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 11:23:12 | classical | INFO: No adversarial input was found for this model for image16.
2022-12-05 11:23:12 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 11:23:12 | classical | INFO: No adversarial input was found for this model for image17.
2022-12-05 11:23:12 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 11:23:12 | classical | INFO: There exists adversarial inputs for this model for image18.
2022-12-05 11:23:12 | classical | INFO: Found 1 adversarial examples so far: [18].
Found 1 adversarial examples so far: [18].
2022-12-05 11:23:12 | 

Out of the 50 images with known adversarial examples, D-Wave's Hybrid solver only found adversarial examples for 17 of these images, with a time limit of 5 seconds per execution.

We repeat this procedure but with a time limit of 10 seconds:

In [1]:
!python quantum/solver_dwave_images_found_T10.py -v

2022-12-05 12:56:16 | classical | DEBUG: Received input argument: Namespace(verbose=True)
2022-12-05 12:56:16 | classical | INFO: No adversarial input was found for this model for image4.
2022-12-05 12:56:16 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 12:56:16 | classical | INFO: No adversarial input was found for this model for image16.
2022-12-05 12:56:16 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 12:56:16 | classical | INFO: No adversarial input was found for this model for image17.
2022-12-05 12:56:16 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 12:56:16 | classical | INFO: There exists adversarial inputs for this model for image18.
2022-12-05 12:56:16 | classical | INFO: Found 1 adversarial examples so far: [18].
Found 1 adversarial examples so far: [18].
2022-12-05 12:56:17 | 

Unfortunately, it seems that doubling the solving time did not improve the results, as it still only found adversarial examples for 17 images.

### Closer Inspection

We decided to check whether the solver would be able to find adversarial examples at the maximum time limit of 60 seconds for three random images: the 4th, the 63rd, and the 206th.

In [4]:
!python quantum/solver_dwave_image_4_T60.py -v

2022-12-05 13:04:33 | classical | DEBUG: Received input argument: Namespace(verbose=True)
Executing LeapHybridCQMSampler with image4
2022-12-05 13:05:59 | classical | INFO: No adversarial input was found for this model for image4.
2022-12-05 13:05:59 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 13:06:00 | classical | INFO: Successfully ran D-Wave's Hybrid CQM Solver for 50 images.


In [5]:
!python quantum/solver_dwave_image_63_T60.py -v

2022-12-05 13:07:35 | classical | DEBUG: Received input argument: Namespace(verbose=True)
Executing LeapHybridCQMSampler with image63
2022-12-05 13:09:01 | classical | INFO: No adversarial input was found for this model for image63.
2022-12-05 13:09:01 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 13:09:02 | classical | INFO: Successfully ran D-Wave's Hybrid CQM Solver for 50 images.


In [6]:
!python quantum/solver_dwave_image_206_T60.py -v

2022-12-05 13:11:54 | classical | DEBUG: Received input argument: Namespace(verbose=True)
Executing LeapHybridCQMSampler with image206
2022-12-05 13:13:19 | classical | INFO: No adversarial input was found for this model for image206.
2022-12-05 13:13:19 | classical | INFO: Found 0 adversarial examples so far: [].
Found 0 adversarial examples so far: [].
2022-12-05 13:13:20 | classical | INFO: Successfully ran D-Wave's Hybrid CQM Solver for 50 images.


Unfortunately, even with 60 seconds of solving time, the solver was not able to find any adversarial examples for these images.

## References

* https://arxiv.org/pdf/1711.07356.pdf
* https://doi.org/10.1007/s10773-019-04379-y
* https://www.dwavesys.com/media/rldh2ghw/14-1055a-a_hybrid_solver_for_constrained_quadratic_models.pdf
* https://www.dwavesys.com/media/0ftjce4l/measuring-performance-of-the-leap-constrained-quadratic-model-solver.pdf
* https://docs.ocean.dwavesys.com/projects/hybrid/en/latest/intro/overview.html
